In [ ]:
from google.colab import drive
drive.mount('/content/drive')
'''
Code inspired from: (They were just inspirations, up to our knowledge, AC_PATE_GAN is not implemented so far.)
https://github.com/BorealisAI/private-data-generation/tree/master/models, 
https://github.com/tensorflow/privacy/tree/master/research/pate_2017,
https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/acgan/acgan.py,
https://github.com/greenelab/SPRINT_gan/blob/master/ac_gan.py
'''

Mounted at /content/drive


# Data

## Load Credit Card Data

Don't run this section. This section is for loading another dataset that we tested. The results were not satisfying, so, we changed the data set.

In [ ]:
from pandas import read_csv
dataframe = read_csv('/content/drive/MyDrive/Privacy/creditcard.csv', header=None)
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
from collections import Counter
# summarize the class distribution
target = dataframe.values[:,-1]
counter = Counter(target)
for k,v in counter.items():
	per = v / len(target) * 100
	print('Class=%d, Count=%d, Percentage=%.3f%%' % (k, v, per))

Class=0, Count=284315, Percentage=99.827%
Class=1, Count=492, Percentage=0.173%


In [ ]:
import numpy as np

train_labels = y.reshape((284807, 1))
COND_num_classes = 2 # Number of classes
train_labels_vec = np.zeros((len(train_labels), COND_num_classes), dtype='float32')
for i, label in enumerate(train_labels):
    train_labels_vec[i, int(train_labels[i])] = 1.0

train_data = X.astype('float32')
print(train_data.shape,train_labels_vec.shape)

NameError: ignored

## Load Banknote  Authentication  Dataset 

In [ ]:
from pandas import read_csv
from scipy.special import expit
from sklearn.utils import shuffle
import pickle


def load_dataset(full_path = '/content/drive/MyDrive/Privacy/real_data_train.csv'):
  # load the dataset as a numpy array
  with open(full_path, 'rb') as f:
    data = pickle.load(f)
  # retrieve numpy array
  data = data.values
  # split into input and output elements
  X, y = data[:, :-1], data[:, -1]
  return X, y

X, y = load_dataset()
# preprocessing
X = expit(X)
X, y = shuffle(X, y)
print(X.shape,y.shape)

# Utils

In [ ]:
# Copyright 2019 RBC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import torch
import torch.nn as nn
import math
import numpy as np
import pandas as pd
from sklearn.metrics import mutual_info_score


class Generator(nn.Module):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : FC1024_BR-FC7x7x128_BR-(64)4dc2s_BR-(1)4dc2s_S
    def __init__(self, latent_size, input_dim):
        
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            
            nn.Linear((1 +latent_size), 2*latent_size),
            nn.ReLU(),
            nn.Linear(2*latent_size, input_dim),
            nn.Sigmoid(),
        )

    def forward(self, input, label):
       
        x = torch.cat([input, label], 1)
        x = self.fc(x)
        return x

class Discriminator(nn.Module):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : (64)4c2s-(128)4c2s_BL-FC1024_BL-FC1_S

    def __init__(self, input_dim):
        super(Discriminator, self).__init__()

        self.fc1 = nn.Sequential(
            nn.Linear(input_dim, 20), 
            nn.ReLU(),
            nn.Linear(20, 16),
            nn.ReLU(),
            nn.Linear(16, 10),
            nn.ReLU(),
        )
        self.dc = nn.Sequential(
            nn.Linear(10, 1),
            nn.Sigmoid(),
        )
        self.cl = nn.Sequential(
            nn.Linear(10, 1),
            nn.Sigmoid(),
        )

    def forward(self, input):
        x = self.fc1(input)
        d = self.dc(x)
        c = self.cl(x)
        return d, c



def weights_init(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


def pate(data, netTD, lap_scale):
    results = torch.Tensor(len(netTD), data.size()[0]).type(torch.int64)
    for i in range(len(netTD)):
        output, aux = netTD[i].forward(data)
        pred = (output > 0.5).type(torch.Tensor).squeeze()
        results[i] = pred

    clean_votes = torch.sum(results, dim=0).unsqueeze(1).type(torch.cuda.DoubleTensor)
    noise = torch.from_numpy(np.random.laplace(loc=0, scale=1/lap_scale, size=clean_votes.size())).cuda()
    noisy_results = clean_votes + noise
    noisy_labels = (noisy_results > len(netTD)/2).type(torch.cuda.DoubleTensor)

    return noisy_labels, clean_votes


def moments_acc(num_teachers, clean_votes, lap_scale, l_list):
    q = (2 + lap_scale * torch.abs(2*clean_votes - num_teachers)
         )/(4 * torch.exp(lap_scale * torch.abs(2*clean_votes - num_teachers)))

    update = []
    for l in l_list:
        a = 2*lap_scale*lap_scale*l*(l + 1)
        t_one = (1 - q) * torch.pow((1 - q) / (1 - math.exp(2*lap_scale) * q), l)
        t_two = q * torch.exp(2*lap_scale * l)
        t = t_one + t_two
        update.append(torch.clamp(t, max=a).sum())

    return torch.cuda.DoubleTensor(update)


def mutual_information(labels_x: pd.Series, labels_y: pd.DataFrame):

    if labels_y.shape[1] == 1:
        labels_y = labels_y.iloc[:, 0]
    else:
        labels_y = labels_y.apply(lambda x: ' '.join(x.get_values()), axis=1)

    return mutual_info_score(labels_x, labels_y)


def normalize_given_distribution(frequencies):
    distribution = np.array(frequencies, dtype=float)
    distribution = distribution.clip(0)  # replace negative values with 0
    summation = distribution.sum()
    if summation > 0:
        if np.isinf(summation):
            return normalize_given_distribution(np.isinf(distribution))
        else:
            return distribution / summation
    else:
        return np.full_like(distribution, 1 / distribution.size)

# Model

## ACPATE GAN

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data_utils
import numpy as np
import math


class PATE_ACGAN:
    def __init__(self, input_dim, z_dim, num_teachers, target_epsilon, target_delta):
        self.generator = Generator(z_dim, input_dim).cuda().double()
        self.student_disc = Discriminator(input_dim).cuda().double()
        self.teacher_disc = [Discriminator(input_dim).cuda().double()
                             for _ in range(num_teachers)]
        self.generator.apply(weights_init)
        self.student_disc.apply(weights_init)
        self.z_dim = z_dim
        self.num_teachers = num_teachers
        for i in range(num_teachers):
            self.teacher_disc[i].apply(weights_init)

        self.target_epsilon = target_epsilon
        self.target_delta = target_delta
        self.conditional = False


    def train(self, x_train, y_train, hyperparams):
        batch_size = hyperparams.batch_size
        num_teacher_iters = hyperparams.num_teacher_iters
        num_student_iters = hyperparams.num_student_iters
        num_moments = hyperparams.num_moments
        lap_scale = hyperparams.lap_scale
        class_ratios = torch.from_numpy(hyperparams.class_ratios)

        real_label = 1
        fake_label = 0

        alpha = torch.cuda.DoubleTensor([0.0 for _ in range(num_moments)])
        l_list = 1 + torch.cuda.DoubleTensor(range(num_moments))
        criterion = nn.BCELoss()
        optimizer_g = optim.Adam(self.generator.parameters(), lr=hyperparams.lr)
        optimizer_sd = optim.Adam(self.student_disc.parameters(), lr=hyperparams.lr)
        optimizer_td = [optim.Adam(self.teacher_disc[i].parameters(), lr=hyperparams.lr
                                   ) for i in range(self.num_teachers)]

        tensor_data = data_utils.TensorDataset(torch.cuda.DoubleTensor(x_train), torch.cuda.DoubleTensor(y_train))
        train_loader = []
        for teacher_id in range(self.num_teachers):
            start_id = teacher_id * len(tensor_data) / self.num_teachers
            end_id = (teacher_id + 1) * len(tensor_data) / self.num_teachers if teacher_id != (
                    self.num_teachers - 1) else len(tensor_data)

            train_loader.append(data_utils.DataLoader(torch.utils.data.Subset( \
                tensor_data, range(int(start_id), int(end_id))), batch_size=batch_size, shuffle=True))

        steps = 0
        epsilon = 0

        # while epsilon < self.target_epsilon:
        while steps <= 1000:

            # train the teacher discriminators
            for t_2 in range(num_teacher_iters):
                for i in range(self.num_teachers):
                    inputs, categories = None, None
                    for b, data in enumerate(train_loader[i], 0):
                        inputs, categories = data
                        break

                    # train teachers with real
                    optimizer_td[i].zero_grad()
                    label = torch.full((inputs.size()[0],), real_label).cuda()
                    output, aux = self.teacher_disc[i].forward(inputs)
                    output = torch.squeeze(output)
                    
                    err_d_real = criterion(output, label.double())
                    # adding losses, and then doing the backpropagation
                    err_d_real += criterion(aux, categories.unsqueeze(1).double())
                    err_d_real.backward()

                    # train teachers with fake
                    z = torch.Tensor(batch_size, self.z_dim).uniform_(0, 1).cuda()
                    label.fill_(fake_label)

                    category = torch.multinomial(class_ratios,  inputs.size()[0], replacement=True).unsqueeze(1).cuda().double()
                    fake = self.generator(z.double(), category)
                    output, aux = self.teacher_disc[i].forward(fake)

                    output = torch.squeeze(output)

                    err_d_fake = criterion(output, label.double())
                    # adding losses, and then doing the backpropagation
                    err_d_fake += criterion(aux, categories.unsqueeze(1).double())
                    err_d_fake.backward()

                    optimizer_td[i].step()

            # train the student discriminator
            for t_3 in range(num_student_iters):
                z = torch.Tensor(batch_size, self.z_dim).uniform_(0, 1).cuda()

                category = torch.multinomial(class_ratios,  inputs.size()[0], replacement=True).unsqueeze(1).cuda().double()
                fake = self.generator(z.double(), category)
               
                predictions, clean_votes = pate(fake.detach(), self.teacher_disc, lap_scale)
                outputs, aux = self.student_disc.forward(fake.detach())

                # update the moments
                alpha = alpha + moments_acc(self.num_teachers, clean_votes, lap_scale, l_list)

                # update student
                err_sd = criterion(outputs, predictions)
                optimizer_sd.zero_grad()

                # adding losses, and then doing the backpropagation
                err_sd += criterion(aux, categories.unsqueeze(1).double())
                err_sd.backward()
                optimizer_sd.step()
            # train the generator
            optimizer_g.zero_grad()
            z = torch.Tensor(batch_size, self.z_dim).uniform_(0, 1).cuda()
            label = torch.full((inputs.size()[0],), real_label).cuda()

            category = torch.multinomial(class_ratios,  inputs.size()[0], replacement=True).unsqueeze(1).cuda().double()
            fake = self.generator(z.double(), category)
            output, aux = self.student_disc.forward(fake)
            
            output = torch.squeeze(output)
            err_g = criterion(output, label.double())

            # adding losses, and then doing the backpropagation
            err_g += criterion(aux, categories.unsqueeze(1).double())
            err_g.backward()
            optimizer_g.step()
            # if steps %1000 == 0:
            #   torch.save(self.generator.state_dict(), "/content/drive/MyDrive/Privacy/model_step%d.h5"%steps)
            #   torch.save(self.generator, "/content/drive/MyDrive/Privacy/entire_model_step%d.h5"%steps)

            # Calculate the privacy cost
            epsilon = min((alpha - math.log(self.target_delta)) / l_list)
            if steps % 100 == 0:
                print("Step : ", steps, "Loss SD : ", err_sd.item(), "Loss G : ", err_g.item(), "Epsilon : ",
                      epsilon.item())

            steps += 1


    def generate(self, num_rows, class_ratios, batch_size=1):
        steps = num_rows // batch_size
        print(steps,num_rows)
        synthetic_data = []
        synthetic_label = []
        class_ratios = torch.from_numpy(class_ratios)
        for step in range(steps):
            noise = torch.randn(batch_size, self.z_dim).cuda()

            category = torch.multinomial(class_ratios, batch_size, replacement=True).unsqueeze(1).cuda().double()
            synthetic = self.generator(noise.double(), category)

            synthetic_data.append(synthetic.cpu().data.numpy())
            synthetic_label.append(category.cpu().data.numpy())
            
        print(len(synthetic_data),len(synthetic_label))
        return synthetic_data, synthetic_label

# Evaulate

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn import preprocessing
from scipy.special import expit
import argparse
import numpy as np
import pandas as pd
import collections
import os


# Loading the data
class_ratios = None
target_variable = 4
X_train = np.array(X)
y_train = np.array(y)

X_train = expit(X_train)
print(X_train.shape)
input_dim = X_train.shape[1]
z_dim = int(input_dim / 4 + 1) if input_dim % 4 == 0 else int(input_dim / 4)



Hyperparams = collections.namedtuple('Hyperarams', 'batch_size num_teacher_iters num_student_iters num_moments lap_scale class_ratios lr')
Hyperparams.__new__.__defaults__ = (None, None, None, None, None, None, None)

num_teachers = 10
target_epsilon = 8
batch_size = 16
student_iters = 5
teacher_iters = 5
num_moments = 100
lap_scale = 0.0001
target_delta = 1e-5
model = PATE_ACGAN(input_dim, z_dim, num_teachers, target_epsilon, target_delta)
model.train(X_train, y_train, Hyperparams(batch_size=batch_size, num_teacher_iters=teacher_iters,
                                          num_student_iters=student_iters, num_moments=num_moments,
                                          lap_scale=lap_scale, class_ratios=class_ratios, lr=1e-4))

(1372, 4)
ghable loss
bade loss
HERE
Step :  0 Loss SD :  1.400318656377046 Loss G :  1.442842657272163 Epsilon :  0.11529085464970228
Step :  100 Loss SD :  1.3764717922792942 Loss G :  1.407643440966949 Epsilon :  0.13145085464970213
Step :  200 Loss SD :  1.3606291744990116 Loss G :  1.3775815986391144 Epsilon :  0.1476108546497025
Step :  300 Loss SD :  1.3723837476480256 Loss G :  1.3808966421692728 Epsilon :  0.16377085464970192
Step :  400 Loss SD :  1.34409529624027 Loss G :  1.3513895162905265 Epsilon :  0.17993085464970032
Step :  500 Loss SD :  1.423035929873897 Loss G :  1.4377663173075572 Epsilon :  0.19609085464969897
Step :  600 Loss SD :  1.3785799556427347 Loss G :  1.375566457466443 Epsilon :  0.2122508546497018
Step :  700 Loss SD :  1.3670074261965661 Loss G :  1.34615356900938 Epsilon :  0.22841085464970465
Step :  800 Loss SD :  1.421447884693504 Loss G :  1.4111774024989487 Epsilon :  0.24422228910496158
Step :  900 Loss SD :  1.398183810847983 Loss G :  1.385168

## Generate New Dataset

In [24]:
import pickle

syn_data, label = model.generate(1000, class_ratios)
syn_data = np.array(syn_data)
label = np.array(label)
# print(syn_data.shape)
# print(label.shape)
syn_data = np.squeeze(syn_data)
label = np.squeeze(label)
print(np.where(label==1))

output = open("/content/drive/MyDrive/Privacy/ACPATE_results/X_ACPATE_NEW_low_epoch.csv", 'wb')
pickle.dump(syn_data, output)
output = open("/content/drive/MyDrive/Privacy/ACPATE_results/Y_ACPATE_NEW_low_epoch.csv", 'wb')
pickle.dump(label, output)
output.close()

1000 1000
1000 1000
(1000, 1, 4)
(1000, 1, 1)
SHAPE AFTER
(1000, 4)
(1000,)
(array([  2,   5,   6,  11,  12,  13,  14,  15,  17,  19,  21,  22,  24,
        25,  26,  27,  28,  29,  32,  33,  35,  39,  42,  43,  45,  47,
        49,  56,  57,  58,  60,  61,  62,  63,  65,  66,  67,  68,  72,
        73,  74,  77,  78,  79,  81,  82,  90,  91,  92,  94,  97,  98,
        99, 100, 103, 104, 105, 106, 107, 110, 112, 115, 116, 117, 119,
       123, 125, 127, 129, 130, 132, 133, 137, 139, 140, 141, 143, 145,
       147, 152, 154, 160, 161, 162, 167, 169, 170, 173, 174, 175, 177,
       179, 180, 181, 182, 186, 187, 189, 190, 191, 194, 195, 196, 197,
       201, 202, 203, 205, 206, 207, 208, 213, 217, 219, 220, 221, 222,
       224, 228, 229, 233, 235, 236, 237, 238, 243, 244, 246, 254, 256,
       263, 265, 271, 275, 276, 278, 280, 281, 282, 284, 286, 287, 288,
       293, 296, 299, 301, 304, 307, 308, 309, 313, 314, 316, 317, 318,
       321, 322, 328, 330, 332, 333, 334, 340, 343, 344, 34

## Load Generated Dataset

In [ ]:
import pickle
with open('/content/drive/MyDrive/Privacy/ACPATE_results/X_ACPATE_NEW_low_epoch.csv', 'rb') as f:
    data = pickle.load(f)
# print(data)

with open('/content/drive/MyDrive/Privacy/ACPATE_results/Y_ACPATE_NEW_low_epoch.csv', 'rb') as f:
    data = pickle.load(f)
# print(data)